In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

## Loading Classical Data

In [2]:
df = pd.read_csv('extract_classical.csv')

C:\Users\Nikhil\AppData\Local\Temp/ipykernel_51404/643371311.py:1: DtypeWarning: Columns (32,33,34,43,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('extract_classical.csv')


In [3]:
df.columns

Index(['ID_Transcode', 'MasterCategory', 'Country', 'Category', 'Manufacturer',
       'Corporation', 'Corporation_recode', 'Product_recode', 'Pack_recode',
       'Molecules_recode', 'ATC4_recode', 'PI_Status', 'NFC123_Recode', 'UNIT',
       'Corporation_Group_1', 'Corporation_Group_2', 'Corporation_Group_3',
       'Product_Group_1', 'Product_Group_2', 'Product_Group_3', 'SU', 'IU',
       'IUe', 'TD', 'MG', 'ML', 'Segment', 'isnew', 'isDS',
       'Corporation_Group_1_C', 'Corporation_Group_2_C',
       'Corporation_Group_3_C', 'Product_Group_1_C', 'Product_Group_2_C',
       'Product_Group_3_C', 'SU_C', 'IU_C', 'TD_C', 'MG_C', 'ML_C',
       'Corrected_who', 'Corrected_when', 'LAUNCH_DATE', 'Isnew_Date',
       'Flashclean_Product_date', 'Flashclean_Pack_date',
       'Flashclean_Strengh_date', 'checked', 'PEq', 'PEq_C', 'MOD', 'MOD_C',
       'IUe_C', 'CPC', 'AVD', 'TC', 'TC_C', 'Launch_Date_Recode',
       'LOE_Date_Recode'],
      dtype='object')

In [4]:
cols = ['Country', 'Corporation', 'Corporation_recode', 'Product_recode', 'Pack_recode',
        'Molecules_recode', 'ATC4_recode', 'NFC123_Recode', 'UNIT',
        'Corporation_Group_1', 'Corporation_Group_2', 'Corporation_Group_3',
        'Product_Group_1', 'Product_Group_2', 'Product_Group_3', 'SU', 'IU',
        'IUe', 'TD', 'MG', 'ML', 'Segment']
df[cols].head()

,Country,Corporation,Corporation_recode,Product_recode,Pack_recode,Molecules_recode,ATC4_recode,NFC123_Recode,UNIT,Corporation_Group_1,...,Product_Group_1,Product_Group_2,Product_Group_3,SU,IU,IUe,TD,MG,ML,Segment
0,CHILE,EUROFARMA CORP,EUROFARMA,PARODOX XR,TABL RECU LP 25MG X 30,PAROXETINE,N06A4,ABC,PACK,EUROFARMA,...,PARODOX XR,PARODOX XR,PARODOX XR,NaN,NaN,NaN,NaN,NaN,NaN,Not coded
1,CHILE,D M PHARMA,URIAGE,URIAGE BARIESUN,P ST/I F50X2 16G X 1,ASCORBIC ACID!BORAGO OFFICINALIS!BUTYROSPERMUM...,D02A0,MYM,PACK,Other Corps,...,OTHER SKIN CARE OTHERS,URIAGE BARIESUN,OTHER SKIN CARE OTHERS,NaN,NaN,NaN,NaN,NaN,NaN,Others
2,CENTRAL AMERICA,PAILL,PAILL,SUDAGRIP ANTIGRIP. (PI+),SUDAGRIP ANTIGRIP. SOBRES 15 G X 25,ASCORBIC ACID!CHLORPHENAMINE!PARACETAMOL!PHENY...,R05B0,DEP,PACK,Other Corps,...,OTHER COLD/FLU IMMUNOSTIMULANTS,SUDAGRIP ANTIGRIP. (PI+),OTHER COLD/FLU IMMUNOSTIMULANTS,NaN,NaN,NaN,NaN,NaN,NaN,Others
3,USA,TEVA PHARM USA,TEVA,AMOX TR/POT CLAVUL,PWD SUSP 200-28.5MG 100ML,AMOXICILLIN!CLAVULANIC ACID,J01C1,DEA,PACK,TEVA,...,AMOX TR/POT CLAVUL,AMOX TR/POT CLAVUL,AMOX TR/POT CLAVUL,NaN,NaN,NaN,NaN,NaN,NaN,Not coded
4,SAUDI ARABIA,MERCK & CO,MERCK & CO,JANUMET XR,F.C.T. 1000/ 100 MG 28,METFORMIN!SITAGLIPTIN,A10N3,BBC,PACK,MERCK & CO,...,JANUMET,JANUVIA F,SITAGLIPTIN,28.0,NaN,NaN,NaN,NaN,NaN,COMBO


In [5]:
df.iloc[0:2,5:8]

,Corporation,Corporation_recode,Product_recode
0,EUROFARMA CORP,EUROFARMA,PARODOX XR
1,D M PHARMA,URIAGE,URIAGE BARIESUN


In [6]:
r, c = df.shape
print(df.shape)

(800000, 59)


In [7]:
round((df[cols].isnull()).sum()*100/r, 2)

Country                  0.00
Corporation              0.55
Corporation_recode       0.00
Product_recode           0.00
Pack_recode              0.00
Molecules_recode         0.00
ATC4_recode              0.00
NFC123_Recode            0.00
UNIT                     0.00
Corporation_Group_1      0.00
Corporation_Group_2    100.00
Corporation_Group_3    100.00
Product_Group_1          0.00
Product_Group_2          0.01
Product_Group_3          0.00
SU                      82.67
IU                      98.46
IUe                     99.91
TD                      97.98
MG                      94.93
ML                      99.40
Segment                  0.00
dtype: float64

### Fill NULL values

In [8]:
df.fillna('', inplace=True)

In [9]:
# check the relevant columns for identifying the product group 3 (active molecule)
# we can consider the columns Corporation_recode, Product_recode, Molecules_recode for identifying the Product_Group_3
cols_i = ['Corporation_recode', 'Product_recode', 'Molecules_recode', 'Product_Group_3']
df[cols_i].head()

,Corporation_recode,Product_recode,Molecules_recode,Product_Group_3
0,EUROFARMA,PARODOX XR,PAROXETINE,PARODOX XR
1,URIAGE,URIAGE BARIESUN,ASCORBIC ACID!BORAGO OFFICINALIS!BUTYROSPERMUM...,OTHER SKIN CARE OTHERS
2,PAILL,SUDAGRIP ANTIGRIP. (PI+),ASCORBIC ACID!CHLORPHENAMINE!PARACETAMOL!PHENY...,OTHER COLD/FLU IMMUNOSTIMULANTS
3,TEVA,AMOX TR/POT CLAVUL,AMOXICILLIN!CLAVULANIC ACID,AMOX TR/POT CLAVUL
4,MERCK & CO,JANUMET XR,METFORMIN!SITAGLIPTIN,SITAGLIPTIN


In [10]:
df[cols_i].iloc[1,2]

'ASCORBIC ACID!BORAGO OFFICINALIS!BUTYROSPERMUM PARKII!PERSEA GRATISSIMA!RETINOL!TITANIUM!VITAMIN E'

### We can't make use of Named Entity Recognition for finding out the Product_Group_3 as the entity takes  different forms in the context

### We need to make use of seq2seq model for recoding and transcoding along with some post processing

In [ ]:
df_new = df[~df['Category'].str.contains('_Boost')]

In [ ]:
df_new[df_new['SU'].isnull()]['MasterCategory'].unique()

In [ ]:
df_new.shape

In [ ]:
r_new, c_new = df_new.shape
round((df_new[cols].isnull()).sum()*100/r_new, 2)

In [ ]:
df[df['SU'].isnull()]['MasterCategory'].unique()

In [ ]:
(df[~df['SU'].isnull()])[cols]

In [ ]:
a = df[df['SU'].isnull()]

In [ ]:
a[a['MasterCategory']=='EP'][cols]

### Word2Vec for Classical data

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
import itertools

reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

In [ ]:
a = one_hot("amazing restaurant",2)

In [ ]:
pack_data = list(df['Pack_recode'])

In [ ]:
pack_data = [text_to_word_sequence(x) for x in pack_data]

In [ ]:
vocab = list(itertools.chain(*pack_data))

In [ ]:
len(vocab)

In [ ]:
vocab = list(set(vocab))
len(vocab)

In [ ]:
vocab_size = 60970
encoded_pack = [one_hot(pack, vocab_size) for pack in pack_data]
print(encoded_pack)

In [ ]:
max_len = 0
for x in encoded_pack:
    len_x = len(x)
    if max_len < len_x:
        max_len = len_x
print(max_len)

In [ ]:
max_length = 25
padded_pack = pad_sequences(encoded_pack, maxlen=max_length, padding='post')
print(padded_pack[0])

In [ ]:
embeded_vector_size = 512

model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length,name="embedding"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
X = padded_reviews
y = sentiment

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
df.columns

In [ ]:
df[~df['NFC123_Recode'].str.contains('\-|UNKNOWN', regex=True)]['NFC123_Recode'].value_counts()

In [ ]:
df_new = df['Corporation_recode'] + ' ' + df['Product_recode'] + ' ' + df['Molecules_recode'] + ' ' + df['Pack_recode']
df_new

In [ ]:
import gensim

In [ ]:
def split_text(text):
    text = text.lower()
    text = re.sub(r'[\(\)]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(',', '.')
    text = re.sub(r'[\.\+\!]', ' ', text)
    # text = text.replace('.', ' ').replace('+', ' ').replace('!', ' ')
    return re.split('[\s\-\/]', text)

In [ ]:
df_new = df_new.apply(str)
df_new = df_new.apply(split_text)

In [ ]:
df_new

In [ ]:
model = gensim.models.Word2Vec(vector_size=64,
    window=3,
    min_count=2,
    workers=4, sg=1
)

In [ ]:
model.build_vocab(df_new, progress_per=1000)

In [ ]:
model.train(df_new, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.wv.most_similar('prati'.lower(), 'ibu')

In [ ]:
model.wv.most_similar('eurofarma',
 'paroxetine')

In [ ]:
df_new.iloc[0]

In [ ]:
model.predict_output_word(['eurofarma',
 'paroxetine'])

### Sequence to Sequence modelling 

In [11]:
# check the relevant columns for identifying the product group 3 (active molecule)
# we can consider the columns Corporation_recode, Product_recode, Molecules_recode for identifying the Product_Group_3
cols_i = ['Corporation_recode', 'Product_recode', 'Molecules_recode', 'Product_Group_3']
df[cols_i].head()

,Corporation_recode,Product_recode,Molecules_recode,Product_Group_3
0,EUROFARMA,PARODOX XR,PAROXETINE,PARODOX XR
1,URIAGE,URIAGE BARIESUN,ASCORBIC ACID!BORAGO OFFICINALIS!BUTYROSPERMUM...,OTHER SKIN CARE OTHERS
2,PAILL,SUDAGRIP ANTIGRIP. (PI+),ASCORBIC ACID!CHLORPHENAMINE!PARACETAMOL!PHENY...,OTHER COLD/FLU IMMUNOSTIMULANTS
3,TEVA,AMOX TR/POT CLAVUL,AMOXICILLIN!CLAVULANIC ACID,AMOX TR/POT CLAVUL
4,MERCK & CO,JANUMET XR,METFORMIN!SITAGLIPTIN,SITAGLIPTIN


In [12]:
from fast_ml.model_development import train_valid_test_split

In [13]:
input_text = df['Corporation_recode'] + ' ' + df['Product_recode'] + ' ' + df['Molecules_recode'].str.replace('!', ' ')
output_text = df['Product_Group_3']

In [14]:
raw_text_df = pd.DataFrame(input_text)
raw_text_df['Product_Group_3'] = output_text
raw_text_df.columns = ['input_text', 'target_text']
raw_text_df.head()

,input_text,target_text
0,EUROFARMA PARODOX XR PAROXETINE,PARODOX XR
1,URIAGE URIAGE BARIESUN ASCORBIC ACID BORAGO OF...,OTHER SKIN CARE OTHERS
2,PAILL SUDAGRIP ANTIGRIP. (PI+) ASCORBIC ACID C...,OTHER COLD/FLU IMMUNOSTIMULANTS
3,TEVA AMOX TR/POT CLAVUL AMOXICILLIN CLAVULANIC...,AMOX TR/POT CLAVUL
4,MERCK & CO JANUMET XR METFORMIN SITAGLIPTIN,SITAGLIPTIN


In [15]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(raw_text_df, target = 'target_text',
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)

In [16]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.head()

,input_text,target_text
76658,XELLIA AMPICILLIN/SULBACTAM AMPICILLIN SULBACTAM,AMPICILLIN/SULBACTAM
22924,BIOGEN ZINBRYTA DACLIZUMAB,ZINBRYTA
573616,RAFFO GARDOTON GLIBENCLAMIDE,GLIBENCLAMIDE
705611,ATLANTIC GRUPA NAT&FORM UNKNOWN,OTHER MUSCLE PAIN ANALGESICS
669306,BIOGROUP PSICOPHYT 19 UNKNOWN,OTHER MOOD ENHANCING PRODUCTS


In [17]:
eval_df = pd.concat([X_train, y_train], axis=1)
eval_df.head()

,input_text,target_text
76658,XELLIA AMPICILLIN/SULBACTAM AMPICILLIN SULBACTAM,AMPICILLIN/SULBACTAM
22924,BIOGEN ZINBRYTA DACLIZUMAB,ZINBRYTA
573616,RAFFO GARDOTON GLIBENCLAMIDE,GLIBENCLAMIDE
705611,ATLANTIC GRUPA NAT&FORM UNKNOWN,OTHER MUSCLE PAIN ANALGESICS
669306,BIOGROUP PSICOPHYT 19 UNKNOWN,OTHER MOOD ENHANCING PRODUCTS


In [18]:
test_df = pd.concat([X_test, y_test], axis=1)
test_df.head()

,input_text,target_text
196937,SCHWABE TARTARUS STIB. DHU ANTIMONY,OTHER OTHER NUTRITIONALS
475377,ERBAGIL OTORIG MENTHOL OZONATED OLIVE OIL SODIUM,OTHER DECONGESTANTS
666670,NATIRIS HYPERIFORCE UNKNOWN,OTHER MOOD ENHANCING PRODUCTS
230449,Other BODIE UNKNOWN,OTHER SKIN CARE OTHERS
284444,BOIRON BOIRON ARNICA MONTANA SARCOLACTICUM,OTHER ANALGESICS - TOPICAL


In [19]:
import logging

import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


'''
train_data = [
    [
        "Perseus “Percy” Jackson is the main protagonist and the narrator of the Percy Jackson and the Olympians series.",
        "Percy is the protagonist of Percy Jackson and the Olympians",
    ],
    [
        "Annabeth Chase is one of the main protagonists in Percy Jackson and the Olympians.",
        "Annabeth is a protagonist in Percy Jackson and the Olympians.",
    ],
]
'''

train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])

'''
eval_data = [
    [
        "Grover Underwood is a satyr and the Lord of the Wild. He is the satyr who found the demigods Thalia Grace, Nico and Bianca di Angelo, Percy Jackson, Annabeth Chase, and Luke Castellan.",
        "Grover is a satyr who found many important demigods.",
    ],
    [
        "Thalia Grace is the daughter of Zeus, sister of Jason Grace. After several years as a pine tree on Half-Blood Hill, she got a new job leading the Hunters of Artemis.",
        "Thalia is the daughter of Zeus and leader of the Hunters of Artemis.",
    ],
]
'''

eval_df = pd.DataFrame(eval_data, columns=["input_text", "target_text"])



In [ ]:
# Configure the model
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 200
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args, use_cuda=False
)

# Train the model
model.train_model(train_df, eval_data=eval_df)

# Evaluate the model
result = model.eval_model(eval_df)



INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/640000 [00:00<?, ?it/s]

In [ ]:
# Use the model for prediction
print(
    model.predict(
        [
            "Tyson is a Cyclops, a son of Poseidon, and Percy Jackson’s half brother. He is the current general of the Cyclopes army."
        ]
    )
)

In [266]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
train_data = [
    ["Aragorn was the heir of Isildur", 1],
    ["Frodo was the heir of Isildur", 0],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_data = [
    ["Theoden was the king of Rohan", 1],
    ["Merry was the king of Rohan", 0],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args, use_cuda=False, fp=False
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
predictions, raw_outputs = model.predict(["Sam was a Wizard"])

ModuleNotFoundError: No module named 'torch'